<a href="https://colab.research.google.com/github/Sidy3143/Reinforcement-Learning/blob/main/Run_shoot_pass.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get update
!apt-get install -y libsdl2-dev libsdl2-image-dev libsdl2-ttf-dev libsdl2-gfx-dev \
    libboost-all-dev cmake libopusfile-dev ffmpeg

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,185 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,619 kB]
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:13 http://security.ubuntu.com/ubuntu jammy-s

In [ ]:
!pip install gfootball==2.9

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 52.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for gfootball: filename=gfootball-2.9-cp310-cp310-linux_x86_64.whl size=31513444 sha256=512cd583c54dab2090065b4afd190e138c3ca16acda3d4eccd652495a410a0c2
  Stored in directory: /root/.cache/pip/wheels/86/3d/28/3671579a7344f4d71808aa3b04be3064ba282dbc8a0602378a
Successfully built gfootball


In [ ]:
!apt-get install xvfb

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libfontenc1 libxfont2 libxkbfile1 x11-xkb-utils xfonts-base xfonts-encodings xfonts-utils
  xserver-common
The following NEW packages will be installed:
  libfontenc1 libxfont2 libxkbfile1 x11-xkb-utils xfonts-base xfonts-encodings xfonts-utils
  xserver-common xvfb
0 upgraded, 9 newly installed, 0 to remove and 56 not upgraded.
Need to get 7,815 kB of archives.
After this operation, 11.9 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 libfontenc1 amd64 1:1.1.4-1build3 [14.7 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 libxfont2 amd64 1:2.0.5-1build1 [94.5 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/main amd64 libxkbfile1 amd64 1:1.1.0-1build3 [71.8 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy/main amd64 x11-xkb-utils amd64 7.7+5build4 [172 kB]
Get:5 http://archiv

In [ ]:
!pip install pyvirtualdisplay

In [ ]:
!pip install stable-baselines3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 958.1/958.1 kB 35.0 MB/s eta 0:00:00


In [ ]:
import torch
import gfootball.env as football_env
from stable_baselines3 import PPO
import os
import imageio
import matplotlib.pyplot as plt
from IPython.display import Image
from stable_baselines3.common.callbacks import BaseCallback
from stable_baselines3.common.vec_env import SubprocVecEnv, DummyVecEnv
from google.colab import drive
import zipfile
import datetime

/usr/local/lib/python3.10/dist-packages/tensorflow/lite/python/util.py:55: DeprecationWarning: jax.xla_computation is deprecated. Please use the AOT APIs; see https://jax.readthedocs.io/en/latest/aot.html. For example, replace xla_computation(f)(*xs) with jit(f).lower(*xs).compiler_ir('hlo'). See CHANGELOG.md for 0.4.30 for more examples.
  from jax import xla_computation as _xla_computation


In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [ ]:
!pip install 'shimmy>=2.0'

In [ ]:
# First, you need to explicitly mount Google Drive and authenticate
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.callbacks import EvalCallback

class TrainingStepLogger(BaseCallback):
    def __init__(self, log_interval=1000, verbose=1):
        super(TrainingStepLogger, self).__init__(verbose)
        self.log_interval = log_interval

    def _on_step(self) -> bool:
        # Log the number of training steps
        print(f"Training step: {self.num_timesteps}")
        if self.n_calls % self.log_interval == 0:
            print(f"Step: {self.n_calls}, Timesteps: {self.num_timesteps}, Reward: {self.locals.get('rewards', None)}")
        return True  # Return True to continue training


In [ ]:
save_dir = "content/drive/MyDrive/models"
os.makedirs(save_dir, exist_ok=True)

env_name = 'academy_run_pass_and_shoot_with_keeper'
logger = TrainingStepLogger(log_interval=1000)

# Create environment
env = football_env.create_environment(
    env_name=env_name,
    representation="simple115",
    rewards='scoring,checkpoints'
)

/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


In [ ]:
# Train with PPO
model = PPO("MlpPolicy", env, verbose=1, device=device)
model.learn(total_timesteps=400000, callback= logger)

model_path = os.path.join('/content/drive/MyDrive/models', 'PPO_run_shot_pass_400000')
os.makedirs(os.path.dirname(model_path), exist_ok=True)
model.save(model_path)

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


/usr/local/lib/python3.10/dist-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/stable_baselines3/common/on_policy_algorithm.py:150: UserWarning: You are trying to run PPO on the GPU, but it is primarily intended to run on the CPU when not using a CNN policy (you are using ActorCriticPolicy which should be a MlpPolicy). See https://github.com/DLR-RM/stable-baselines3/issues/1245 for more info. You can pass `device='cpu'` or `export CUDA_VISIBLE_DEVICES=` to force using the CPU.Note: The model will train, but the GPU utilization will be poor and the training might take longer than on CPU.
  warnings.warn(


Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
Training step: 396477
Training step: 396478
Training step: 396479
Training step: 396480
Training step: 396481
Training step: 396482
Training step: 396483
Training step: 396484
Training step: 396485
Training step: 396486
Training step: 396487
Training step: 396488
Training step: 396489
Training step: 396490
Training step: 396491
Training step: 396492
Training step: 396493
Training step: 396494
Training step: 396495
Training step: 396496
Training step: 396497
Training step: 396498
Training step: 396499
Training step: 396500
Training step: 396501
Training step: 396502
Training step: 396503
Training step: 396504
Training step: 396505
Training step: 396506
Training step: 396507
Training step: 396508
Training step: 396509
Training step: 396510
Training step: 396511
Training step: 396512
Training step: 396513
Training step: 396514
Training step: 396515
Training step: 396516
Training step: 396517
Training step: 39651

In [ ]:
import numpy as np
import imageio
from gym.wrappers import record_video
from IPython.display import HTML, display
from base64 import b64encode
from IPython import get_ipython

# Create directory for videos if it doesn't exist
video_dir = "/content/PPO_run_pass_shoot_videos"
if not os.path.exists(video_dir):
    os.makedirs(video_dir)


env = record_video.RecordVideo(env, video_dir, episode_trigger=lambda episode_id: True)

# Function to show video in Colab
def show_video(video_path):
    mp4 = open(video_path, 'rb').read()
    base64_encoded = b64encode(mp4).decode()
    return HTML(f'<video width=600 controls><source src="data:video/mp4;base64,{base64_encoded}" type="video/mp4"></video>')


# Run episodes and record
scores = []
episodes = 3
game_score=0
for episode in range(episodes):
    obs = env.reset()
    done = False
    t = 0
    rewards = 0
    frames = []
    printed_obs = False

    while not done:
        action, _states = model.predict(obs, deterministic=True)
        obs, reward, done, info = env.step(action)
        rewards += reward

        #print(obs[97:108])
        # print(obs[:22])
        # print(obs[22:44])


        # Capture the frame and append to list
        frame = env.render(mode='rgb_array')
        frames.append(frame)


        if done:
          print(f"Episode {episode} finished after {t+1} timesteps.")
          #print(f"Score:", game_score)
          print(f"reward: ", reward)

        t += 1

    scores.append(rewards)
    if (episode + 1) % 100 == 0:
        print(f"Episode {episode} avg_score {np.mean(scores[-100:])}")

    # Save the frames as a video
    video_path = os.path.join(video_dir, f"episode_3vs1{episode}.mp4")
    imageio.mimsave(video_path, frames, fps=30)

env.close()

# Display the most recent video
video_files = [os.path.join(video_dir, f) for f in os.listdir(video_dir) if f.endswith('.mp4')]
if video_files:
    latest_video = max(video_files, key=os.path.getctime)
    display(show_video(latest_video))

/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/record_video.py:78: UserWarning: WARN: Overwriting existing videos at /content/PPO_run_pass_shoot_videos folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/monitoring/video_recorder.py:89: UserWarning: WARN: Disabling video recorder because environment <GetStateWrapper<SingleAgentRewardWrapper<SingleAgentObservationWrapper<Simple115StateWrapper<CheckpointRewardWrapper<FootballEnv instance>>>>>> was not initialized with any compatible video mode between `single_rgb_array` and `rgb_array`
  logger.warn(
/usr/local/lib/python3.10/dist-pa

Episode 0 finished after 53 timesteps.
reward:  0.0
Episode 1 finished after 53 timesteps.
reward:  0.0
Episode 2 finished after 53 timesteps.
reward:  0.0


In [ ]:
!pip install moviepy

In [ ]:
from moviepy.editor import VideoFileClip, concatenate_videoclips
clips = [VideoFileClip(f) for f in video_files]
final_clip = concatenate_videoclips(clips)

In [ ]:
output_dir = "/content/drive/MyDrive/"
os.makedirs(output_dir, exist_ok=True)
temp_video_path = os.path.join(output_dir, f"concatenated_video_{env_name}.mp4")

final_clip.write_videofile(temp_video_path)
display(show_video(temp_video_path))